<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Make a couple of different EOR sky models for testing: one that is upscaled to Nside=256, and one where the pixels are rotated arbitrarily

In [60]:
from pyradiosky import SkyModel
import healpy as hp
from pathlib import Path
import numpy as np
import h5py
from astropy import units as un
from astropy.coordinates import spherical_to_cartesian, cartesian_to_spherical, SkyCoord

In [26]:
sm = Path("../sky_models")
eor = sm / 'eor'

In [28]:
first_sky.frame

'icrs'

In [36]:
out_params = {
    "component_type": "healpix",
    "nside": 256,
    "hpx_inds": np.arange(hp.nside2npix(256)),
    "hpx_order": "ring",
    "spectral_type": "full",
    #"freq_array": freq,
    #"stokes": stokes,
    "frame": 'icrs',
}


In [29]:
first_sky.write_skyh5?

In [30]:
(sm / 'eor256').mkdir()

In [40]:
stokes256 = np.zeros((4, 1, hp.nside2npix(256))) * first_sky.stokes.unit

for ch in range(175, 325):
    with h5py.File(eor / f"fch0{ch}.skyh5", 'r') as fl:
        mp = fl['Data']['stokes'][0,0]
        freq = fl['Header']['freq_array'][()] * un.Hz
        stokes256[0] = hp.ud_grade(mp, nside_out=256) * first_sky.stokes.unit
        
        sky_model = SkyModel(freq_array=freq, stokes=stokes256, **out_params)    
        sky_model.write_skyh5(sm / 'eor256' / f"fch0{ch}.skyh5")

In [41]:
from scipy.spatial.transform import Rotation

In [43]:
r = Rotation.from_euler('xyz', [np.pi/6, np.pi/10.7, np.pi/17.8])

In [45]:
spherical_to_cartesian?

In [46]:
skypoint = first_sky.copy()

In [47]:
skypoint.healpix_to_point()

In [54]:
skypoint.skycoord

<SkyCoord (ICRS): (ra, dec) in deg
    [( 45.,  89.6345165), (135.,  89.6345165), (225.,  89.6345165), ...,
     (135., -89.6345165), (225., -89.6345165), (315., -89.6345165)]>

In [62]:
(sm / 'eor-rotated').mkdir()

In [63]:
stokes = np.zeros((4, 1, hp.nside2npix(128))) * first_sky.stokes.unit

for ch in range(175, 325):
    sky = SkyModel.from_skyh5(eor / f"fch0{ch}.skyh5")
    sky.healpix_to_point()
    
    lon, lat = sky.get_lon_lat()
    x,y,z = spherical_to_cartesian(r=1, lat=lat, lon=lon)
    x,y,z = r.apply(np.array([x,y,z]).T).T
    _, lat, lon = cartesian_to_spherical(x,y,z)
    
    sky.skycoord = SkyCoord(lon, lat, frame='icrs')
    sky.write_skyh5(sm / 'eor-rotated' / f"fch0{ch}.skyh5")

In [9]:
lon, lat = first_sky.get_lon_lat()

In [14]:
lon.rad

array([0.78539816, 2.35619449, 3.92699082, ..., 2.35619449, 3.92699082,
       5.49778714])

In [15]:
lon = lon.rad
lat = lat.rad

In [17]:
stokes = first_sky.stokes[0,0]

In [27]:
hp.ud_grade?

In [21]:
stokes256 = hp.ud_grade(stokes, nside_out=256)

In [22]:
stokes256.shape

(786432,)